In [4]:
import datetime as dt
import gettsim as gt
import pandas as pd

from gettsim.config import ROOT_DIR
from gettsim.tests.test_zu_versteuerndes_eink import INPUT_COLS

In [5]:
df = pd.read_csv(ROOT_DIR / "tests" / "test_data" / "test_dfs_zve.csv", usecols=INPUT_COLS).query("jahr == 2018")

In [6]:
policy_date = dt.date(2018, 1, 1)
params_dict, policy_func_dict = gt.get_policies_for_date(
    policy_date=policy_date,
    groups=["eink_st_abzuege", "soz_vers_beitr", "kindergeld"],
)

user_columns = [
    "ges_krankenv_beitr_m",
    "arbeitsl_v_beitr_m",
    "pflegev_beitr_m",
    "rentenv_beitr_m",
]
result = gt.compute_taxes_and_transfers(
    df,
    user_columns=user_columns,
    user_functions=policy_func_dict,
    targets=column,
    params=params_dict,
)

NameError: name 'column' is not defined